In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import datetime

import torch

import torch.nn as nn

from src.consts import *
from src.main import main, setup_torch, get_corpus
from src.model import RNNModel
from src.training import train, evaluate
from src.split_cross_entropy_loss import SplitCrossEntropyLoss

from notebooks.utils import summary, check_cuda_mem

In [3]:
use_data_paralellization = False

In [6]:
setup_torch()
device = torch.device("cuda" if USE_CUDA else "cpu")
corpus = get_corpus()

# TODO remove these two lines
assert len(corpus.dictionary) == 602755
assert corpus.valid.size()[0] == 11606861

ntokens = len(corpus.dictionary)
model = RNNModel(MODEL_TYPE, ntokens, EMBEDDINGS_SIZE, HIDDEN_UNIT_COUNT, LAYER_COUNT, DROPOUT_PROB,
                 TIED)
if use_data_paralellization or USE_DATA_PARALLELIZATION:
    model = CustomDataParallel(model)
else:
    model.to(device)
# criterion = nn.CrossEntropyLoss()
criterion = SplitCrossEntropyLoss(EMBEDDINGS_SIZE, splits=[3800, 30000, 180000], verbose=False)
criterion.to(device)

SplitCrossEntropyLoss()

In [7]:
train(model, corpus, criterion, device)

RuntimeError: CUDA out of memory. Tried to allocate 1.14 GiB (GPU 0; 10.91 GiB total capacity; 8.60 GiB already allocated; 948.56 MiB free; 2.17 MiB cached)

In [5]:
timestamp = datetime.datetime.now()
with open(MODEL_FILE_NAME.format(timestamp), 'wb') as f:
    torch.save(model, f)

In [6]:
# with open(MODEL_FILE_NAME.format(timestamp), 'rb') as f:
# with open('models/trained_models/model-2019-05-22 18:36:17.481233.pt', 'rb') as f:
with open('models/trained_models/model-2019-05-23 18:32:48.721073.pt', 'rb') as f:
    model = torch.load(f)
    # after load the rnn params are not a continuous chunk of memory
    # this makes them a continuous chunk, and will speed up forward pass
    model.rnn.flatten_parameters()

In [7]:
val_loss = evaluate(model, corpus, criterion, device)

> /media/gabrielamelo/Novo volume/Projects/portuguese_wsc/src/training.py(72)evaluate()
     71             import ipdb; ipdb.set_trace()
---> 72             loss = criterion(output.view(-1, ntokens), targets)
     73             total_loss += len(data) * loss.item()



ipdb>  output.to(torch.device("cuda:1"))


*** RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable


ipdb>  torch.cuda.device_count()


2


...

torch.Size([35, 10, 602755])


ipdb>  output[0]


*** RuntimeError: cuda runtime error (46) : all CUDA-capable devices are busy or unavailable at /pytorch/aten/src/THC/THCGeneral.cpp:405


ipdb>  c


RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/ATen/native/cuda/SoftMax.cu:545

In [17]:
# /pytorch/aten/src/THC/THCTensorIndex.cu:362: void indexSelectLargeIndex(TensorInfo<T, IndexType>, 
# TensorInfo<T, IndexType>, TensorInfo<long, IndexType>, int, int, IndexType, IndexType, long) 
# [with T = float, IndexType = unsigned int, DstDim = 2, SrcDim = 2, IdxDim = -2, IndexIsMajor = true]: 
# block: [90,0,0], thread: [80,0,0] Assertion `srcIndex < srcSelectDimSize` failed.